In [1]:
## Libraries
import pandas as pd
import sqlite3
import PyPDF2
import os
import numpy as np
from IPython.display import HTML
from IPython.core.interactiveshell import InteractiveShell

In [3]:
### Changing working directory
##Desktop
os.chdir("C:/Users/john/Documents/GitHub_Data/VCAP-NC-Shelter-Reimbursement-Project-Data")

##laptop
#os.chdir("C:/Users/johnc/Documents/GitHub_Data/VCAP-NC-Shelter-Reimbursement-Project-Data")

In [4]:
### Reading in PDF Data
from PyPDF2 import PdfReader

reader = PdfReader("Summaryreport2001fromDHHS.pdf")
#page = reader.pages[0]
#print(page.extract_text())

reader.pages[1].extract_text()

'Full name Asheville Humane Society\nDepartment\n72 Lees Creek Rd\nAsheville NC 28806\nspecies # into shelter # adopted out # returned to owner # euthanizedtotal operating expenses\ncost per animal handled$1,258,398.00\n$75.68 ID # 7\nbird 60 15 0 13\ncat 4155 660 74 3193\ndog 5080 1259 615 3037\nferret 12 4 0 4\nfish 29 1 20 0\nfox 6 0 0 5\ngerbil 23 6 0 22\ngoat 8 1 0 1\ngroundhog 1 0 0 1\nguinea pig 28 10 0 15\nhamster 22 2 0 15\nlivestock 14 0 10 0\nmouse 24 10 0 16\nopossum 16 0 0 14\npotbelly pig 4 1 0 1\npython 1 1 0 0\nrabbit 39 8 0 35\nraccoon 74 0 0 69\nsalamander 0 1 0 0\nskunk 2 0 0 2\nsquirrel 7 0 0 5\ntarantula 0 1 0 0\nTuesday, January 03, 2012 Page 2 of 34'

In [14]:
from PyPDF2 import PdfReader

def combinePDFPages(pdf_file, date_str, num_of_page_str):
    reader = PdfReader(pdf_file)

    final_page = False
    page_num = 0
    combined_pdf = ""
    try:
        while final_page == False:
            pdf_text = reader.pages[page_num].extract_text()
            pdf_text = pdf_text.replace(date_str, "")
            pdf_text = pdf_text.replace(num_of_page_str, "")
            pdf_text = pdf_text.replace("Page" + " " + str(page_num + 1), "")
            pdf_text = pdf_text.replace("\n  ", "\n")
            combined_pdf += pdf_text
            page_num += 1
    except IndexError:
        final_page = True

    return combined_pdf

In [20]:
pdf_2001_combined = combinePDFPages("Summaryreport2001fromDHHS.pdf", "Tuesday, January 03, 2012", "of 34")
pdf_2002_combined = combinePDFPages("Summaryreport2002fromDHHS.pdf", "03-Jan-12", "of 47")
pdf_2003_combined = combinePDFPages("Summaryreport2003fromDHHS.pdf", "03-Jan-12", "of 43")
pdf_2004_combined = combinePDFPages("Summaryreport2004fromDHHS.pdf", "03-Jan-12", "of 50")
pdf_2005_combined = combinePDFPages("Summaryreport2005fromDHHS.pdf", "03-Jan-12", "of 42")
pdf_2006_combined = combinePDFPages("Summaryreport2006fromDHHS.pdf", "03-Jan-12", "of 31")
pdf_2007_combined = combinePDFPages("Summaryreport2007fromDHHS.pdf", "03-Jan-12", "of 29")
pdf_2008_combined = combinePDFPages("Summaryreport2008fromDHHS.pdf", "03-Jan-12", "of 34")
pdf_2009_combined = combinePDFPages("Summaryreport2009fromDHHS.pdf", "03-Jan-12", "of 37")

### Creating Functions for first PDF

In [31]:
pdf_2002_combined

'Animal Shelter Reporting Data\nfor calendar year 2002Data required to be reported by animal shelters \nmeeting criteria set forth by the "Spay/Neuter Law"\nNC animal shelter reporting data, calendar year 2002, per "Spay/Neuter Law"\nAlleghany County Animal Control\nco/ Public Works\nPO Box 366\nSparta NC 28675\nspecies # into shelter # adopted out # returned to owner # euthanizedtotal operating expenses\ncost per animal handled$11,200.00\n$22.31ID # 4\ncat 55 43 2 10\ndog 447 254 12 178\nAnson County Animal Control\nAnson County Health Dept\nPO Box 473\nWadesboro NC 28170\nspecies # into shelter # adopted out # returned to owner # euthanizedtotal operating expenses\ncost per animal handled$34,759.64\n$27.48ID # 5\ncat 315 0 0 315\ndog 950 0 6 944\nAshe County Animal Control\nPO Box 1797\nWest Jefferson NC 28694\nspecies # into shelter # adopted out # returned to owner # euthanizedtotal operating expenses\ncost per animal handled$114,118.00\n$78.91ID # 6\ncat 705 33 19 653\ndog 741 84 

In [34]:
def returnNumTables(combined_pdf):
    return (len(combined_pdf.split("Full name")) - 1)
    

In [96]:
def addShelterInfoData(combined_pdf, num_tables, my_dict):
    for i in range(1,num_tables + 1):
        my_dict["table_num"].append(i)
        my_dict["lic_id"].append(combined_pdf.split("total operating expenses\ncost per animal handled")[i].split("\n")[1].lstrip().rstrip().split(" ")[-1])
        my_dict["year"].append(2001)
        
        try:
            if len(combined_pdf.split("Department")[i].split("\n")[2].split(" ")) > 3:
                county_city = " ".join(combined_pdf.split("Department")[i].split("\n")[2].split(" ")[0:len(combined_pdf.split("Department")[i].split("\n")[2].split(" "))-2])
                my_dict["city/county"].append(county_city)
                
            
            else:
                my_dict["city/county"].append(combined_pdf.split("Department")[i].split("\n")[2].split(" ")[0])
            
        except IndexError:
            
            my_dict["city/county"].append("Error")
            
        try:
            my_dict["state"].append(combined_pdf.split("Department")[i].split("\n")[2].split(" ")[-2])
        
        except IndexError:
            my_dict["state"].append("Error")
            
        try:
            my_dict["zip"].append(combined_pdf.split("Department")[i].split("\n")[2].split(" ")[-1])
        
        except IndexError:
            my_dict["zip"].append("Error")
            
        my_dict["facility_name"].append(combined_pdf.split("Full name")[i].split("\n")[0].lstrip().rstrip())
    return my_dict

In [85]:
def addExpensesData(combined_pdf, num_tables, my_dict):
    for i in range(1,num_tables + 1):
        
        my_dict["operating_expense"].append(combined_pdf.split("total operating expenses\ncost per animal handled")[i].split("\n")[0].lstrip().rstrip())
        my_dict["cost_per_animal"].append(combined_pdf.split("total operating expenses\ncost per animal handled")[i].split("\n")[1].lstrip().rstrip().split(" ")[0])
        
    return my_dict

In [110]:
### ONly add columns needed for unique idnetification then I cna do a join to get the other tables. That will make the function cleaner.
def addSpeciesData(combined_pdf, num_tables, my_dict):
    for i in range(1,num_tables + 1):
        dat_list = pdf_2001_combined.split("species # into shelter # adopted out # returned to owner # euthanizedtotal operating expenses\ncost per animal handled")[i].rstrip().lstrip().split('\n')
        species_num = 2
        end_table = False
        
        try: 
            while end_table == False:
                if "Full" in dat_list[species_num] or "no report" in dat_list[species_num]:
                    end_table = True
                else:
                    species_and_counts = dat_list[species_num].split(" ")
                    my_dict["table_num"].append(i)
                    
                    
                    if len(species_and_counts[0].split(" ")) > 1:
                        
                        
                    my_dict["species"].append(species_and_counts[0])
                    
                    
                    my_dict["intake"].append(species_and_counts[-4])
                    my_dict["adopted"].append(species_and_counts[-3])
                    my_dict["returned"].append(species_and_counts[-2])
                    my_dict["euthanized"].append(species_and_counts[-1])  
                    species_num += 1
                    
            
        except IndexError:
            my_dict["table_num"].append(i)
            my_dict["species"].append("Error")
            my_dict["intake"].append("Error")
            my_dict["adopted"].append("Error")
            my_dict["returned"].append("Error")
            my_dict["euthanized"].append("Error")
            species_num += 1
    return my_dict

In [97]:
empty_info_2001_dict = {"table_num":[], "lic_id":[], "year":[], "city/county":[], "state":[], "zip":[], "facility_name":[]}  

In [87]:
empty_expenses_2001_dict = {"operating_expense":[], "cost_per_animal":[]}

In [111]:
empty_species_2001_dict = {"table_num":[], "species":[], "intake":[], "adopted":[], "returned":[], "euthanized":[]}

In [98]:
nc_shelter_info_2001_df = addShelterInfoData(pdf_2001_combined, 93, empty_info_2001_dict)

In [89]:
nc_expenses_2001_df = addExpensesData(pdf_2001_combined, 93, empty_expenses_2001_dict)

In [112]:
nc_species_2001_df = addSpeciesData(pdf_2001_combined, 93, empty_species_2001_dict)

In [84]:
print(len(nc_shelter_info_2001_df["lic_id"]))
print(len(nc_shelter_info_2001_df["year"]))
print(len(nc_shelter_info_2001_df["city/county"]))
print(len(nc_shelter_info_2001_df["state"]))
print(len(nc_shelter_info_2001_df["zip"]))
print(len(nc_shelter_info_2001_df["facility_name"]))

93
93
93
93
93
93


In [91]:
print(len(nc_expenses_2001_df["operating_expense"]))
print(len(nc_expenses_2001_df["cost_per_animal"]))

93
93


In [121]:
last_two_elements = nc_species_2001_df['table_num'][:]
print(last_two_elements)

print(nc_species_2001_df['intake'][:])

[1, 1, 1, 2, 2, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 6, 6, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 14, 14, 15, 15, 15, 15, 15, 15, 15, 16, 16, 16, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 20, 20, 21, 21, 21, 21, 21, 21, 21, 21, 23, 23, 24, 24, 24, 24, 24, 25, 25, 25, 25, 25, 25, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 29, 29, 29, 29, 30, 30, 30, 31, 31, 32, 32, 32, 32, 32, 32, 32, 34, 34, 34, 34, 34, 34, 34, 34, 34, 35, 35, 36, 36, 37, 37, 37, 38, 38, 38, 38, 38, 38, 38, 42, 42, 43, 44, 44, 45, 45, 45, 45, 45, 47, 47, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 49, 49, 49, 49, 49, 49, 49, 49, 50, 50, 

In [ ]:
nc_shelter_info_2001_df

In [142]:
nc_shelter_expenses_2001_dict = addExpensesData(93, empty_expenses_2001_dict)

In [143]:
nc_shelter_species_2001_dict = addSpeciesData(93, empty_species_2001_dict)

['$9,826.85', '$19.23 ID # 4', 'cat 50 44 0 6', 'dog 460 187 33 240', 'equine 1 0 1 0', 'Full name Anson County Animal Control', 'Department Anson County Health Dept', 'PO Box 473', 'Wadesboro NC 28170']
['$39,335.00', '$29.44 ID # 5', 'cat 450 0 1 449', 'dog 886 0 11 875', 'Full name Ashe County Animal Control', 'Department', 'PO Box 1797', 'West Jefferson NC 28694']
['$82,883.52', '$66.36 ID # 6', 'cat 560 35 3 522', 'dog 689 78 40 571', 'Full name Asheville Humane Society', 'Department', '72 Lees Creek Rd', 'Asheville NC 28806']
['$1,258,398.00', '$75.68 ID # 7', 'bird 60 15 0 13', 'cat 4155 660 74 3193', 'dog 5080 1259 615 3037', 'ferret 12 4 0 4', 'fish 29 1 20 0', 'fox 6 0 0 5', 'gerbil 23 6 0 22', 'goat 8 1 0 1', 'groundhog 1 0 0 1', 'guinea pig 28 10 0 15', 'hamster 22 2 0 15', 'livestock 14 0 10 0', 'mouse 24 10 0 16', 'opossum 16 0 0 14', 'potbelly pig 4 1 0 1', 'python 1 1 0 0', 'rabbit 39 8 0 35', 'raccoon 74 0 0 69', 'salamander 0 1 0 0', 'skunk 2 0 0 2', 'squirrel 7 0 0 5

IndexError: list index out of range

In [121]:
nc_shelter_expenses_2001_df = pd.DataFrame(nc_shelter_expenses_2001_dict)

In [122]:
nc_shelter_expenses_2001_df.head()

,lic_id,year,city/county,state,zip,facility_name,operating_expense,cost_per_animal
0,4,2001,Sparta,NC,28675,Alleghany County Animal Control,"$9,826.85",$19.23
1,5,2001,Wadesboro,NC,28170,Anson County Animal Control,"$39,335.00",$29.44
2,6,2001,West Jefferson,NC,28694,Ashe County Animal Control,"$82,883.52",$66.36
3,7,2001,Asheville,NC,28806,Asheville Humane Society,"$1,258,398.00",$75.68
4,9,2001,Washington,NC,27889,Beaufort County Animal Control,"$117,529.00",$21.46


In [130]:
nc_shelter_species_2001_df = pd.DataFrame(nc_shelter_species_2001_dict)

In [131]:
nc_shelter_species_2001_df.head()

,lic_id,year,city/county,state,zip,facility_name
0,4,2001,Sparta,NC,28675,Alleghany County Animal Control
1,5,2001,Wadesboro,NC,28170,Anson County Animal Control
2,6,2001,West Jefferson,NC,28694,Ashe County Animal Control
3,7,2001,Asheville,NC,28806,Asheville Humane Society
4,9,2001,Washington,NC,27889,Beaufort County Animal Control


In [70]:
### This parses out the name of the animal shelter
combined_pdf.split("Full name")[1].split("\n")[0].lstrip().rstrip()

'Alleghany County Animal Control'

In [49]:
combined_pdf.split("Department")[17].split("\n")[2].split(" ")[0]

'Burlington'

In [97]:
#This parses out the city, state, and zip
if len(combined_pdf.split("Department")[3].split("\n")[2].split(" ")) > 3:
    county_city = " ".join(combined_pdf.split("Department")[3].split("\n")[2].split(" ")[0:len(combined_pdf.split("Department")[3].split("\n")[2].split(" "))-2])
    print(county_city)


West Jefferson


In [12]:
###This parses out the department
combined_pdf.split("Department")[1].split("\n")[0].lstrip().rstrip()

'co/ Public Works'

In [13]:
#This parses out the total operating expense
combined_pdf.split("total operating expenses\ncost per animal handled")[1].split("\n")[0].lstrip().rstrip()

'$9,826.85'

In [14]:
#This block parses out the cost per animal
combined_pdf.split("total operating expenses\ncost per animal handled")[1].split("\n")[1].lstrip().rstrip().split(" ")[0]

'$19.23'

In [15]:
###This Parses out the ID number
combined_pdf.split("total operating expenses\ncost per animal handled")[1].split("\n")[1].lstrip().rstrip().split(" ")[-1]

'4'

In [95]:
###This parses out species infromation
for i in range(1,94):
    dat_list = pdf_2001_combined.split("species # into shelter # adopted out # returned to owner # euthanizedtotal operating expenses\ncost per animal handled")[i].rstrip().lstrip().split('\n')
    species_num = 2
    end_table = False
    print(table_num)
    print(dat_list)
    try: 
        while end_table == False:
            if "Full" in dat_list[species_num] or "no report" in dat_list[species_num]:
                end_table = True
            else:
                species_and_counts = dat_list[species_num].split(" ")
                species = species_and_counts[0]
                intake = species_and_counts[1]
                adopted = species_and_counts[2]
                returned = species_and_counts[3]
                euthanized = species_and_counts[4]  
                species_num += 1
                print(species, intake, adopted,returned, euthanized)
            
    except IndexError:
        species = "Error"
        intake = "Error"
        adopted = "Error"
        returned = "Error"
        euthanized = "Error"
        print(species, intake, adopted,returned, euthanized)
        
        species_num += 1
                        

399277
['$9,826.85', '$19.23 ID # 4', 'cat 50 44 0 6', 'dog 460 187 33 240', 'equine 1 0 1 0', 'Full name Anson County Animal Control', 'Department Anson County Health Dept', 'PO Box 473', 'Wadesboro NC 28170']
cat 50 44 0 6
dog 460 187 33 240
equine 1 0 1 0
399277
['$39,335.00', '$29.44 ID # 5', 'cat 450 0 1 449', 'dog 886 0 11 875', 'Full name Ashe County Animal Control', 'Department', 'PO Box 1797', 'West Jefferson NC 28694']
cat 450 0 1 449
dog 886 0 11 875
399277
['$82,883.52', '$66.36 ID # 6', 'cat 560 35 3 522', 'dog 689 78 40 571', 'Full name Asheville Humane Society', 'Department', '72 Lees Creek Rd', 'Asheville NC 28806']
cat 560 35 3 522
dog 689 78 40 571
399277
['$1,258,398.00', '$75.68 ID # 7', 'bird 60 15 0 13', 'cat 4155 660 74 3193', 'dog 5080 1259 615 3037', 'ferret 12 4 0 4', 'fish 29 1 20 0', 'fox 6 0 0 5', 'gerbil 23 6 0 22', 'goat 8 1 0 1', 'groundhog 1 0 0 1', 'guinea pig 28 10 0 15', 'hamster 22 2 0 15', 'livestock 14 0 10 0', 'mouse 24 10 0 16', 'opossum 16 0 0 